In [1]:
import torch
import torch.nn.functional as F

from torch_geometric.nn import GCNConv, VGAE
from torch_geometric.loader import DataLoader

from torchvision import transforms

from lib.lib import SignatureDataset, image_to_graph

from tqdm import tqdm

In [2]:
def transform(**kwargs):
    return transforms.Compose([
        transforms.Grayscale(num_output_channels=kwargs['num_output_channels']),
        transforms.Resize(kwargs['resize']),
        transforms.ToTensor(),
    ])

dataset = SignatureDataset(
    root_dir="test_image",
    transform=transform(num_output_channels=1, resize=(150, 150))
)

Loaded 4 signature images (genuine + forged)


In [3]:
class GNNEncoder(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, latent_dim):
        super(GNNEncoder, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv_mu = GCNConv(hidden_channels, latent_dim)
        self.conv_logvar = GCNConv(hidden_channels, latent_dim)

    def forward(self, x, edge_index):
        # Step 1: Aggregate node features from neighbors
        x = F.relu(self.conv1(x, edge_index))

        # Step 2: Output mean and log variance
        mu = self.conv_mu(x, edge_index)
        logvar = self.conv_logvar(x, edge_index)

        return mu, logvar

In [4]:
test_graph = []

# Convert training dataset
for t in tqdm(dataset, desc="Train Graphs", leave=False):
    t_graph = image_to_graph(t)
    test_graph.append(t_graph)

print("Train graphs:", len(test_graph))

Train graphs: 4


In [5]:
test_loader = DataLoader(
    test_graph,
    batch_size=3,        # Process 32 graphs at once
    shuffle=True,         # Shuffle for training
    num_workers=0,        # 0 for debugging, 2-4 for faster loading
    # pin_memory=True 
)

next(iter(test_loader))

DataBatch(x=[3072, 3], edge_index=[2, 11904], batch=[3072], ptr=[4])

In [6]:
input_dim = next(iter(test_graph)).x.shape[1]
hidden_dim = 64
latent_dim = 128

In [7]:
model = VGAE(GNNEncoder(in_channels=input_dim, hidden_channels=hidden_dim, latent_dim=latent_dim))
model.load_state_dict(torch.load('VGAE_Model.pt', weights_only=True))
model.eval()

VGAE(
  (encoder): GNNEncoder(
    (conv1): GCNConv(3, 64)
    (conv_mu): GCNConv(64, 128)
    (conv_logvar): GCNConv(64, 128)
  )
  (decoder): InnerProductDecoder()
)

In [12]:
data = test_graph[3]
data

Data(x=[1024, 3], edge_index=[2, 3968])

In [13]:
with torch.no_grad():
    z = model.encode(data.x, data.edge_index)

z

tensor([[-1.9273e-02, -2.4731e-01,  5.1219e-01,  ...,  4.7546e-01,
         -6.1950e-01,  3.1648e-01],
        [-1.0733e-02, -2.6901e-01,  5.3688e-01,  ...,  5.0413e-01,
         -6.6039e-01,  3.3467e-01],
        [-4.5014e-04, -2.6558e-01,  5.1661e-01,  ...,  4.8351e-01,
         -6.2979e-01,  3.1698e-01],
        ...,
        [-5.9373e-01, -1.6010e-01,  7.3992e-01,  ...,  5.1322e-01,
          4.7961e-01,  1.1018e-01],
        [-6.0516e-01, -1.7531e-01,  7.6734e-01,  ...,  5.3392e-01,
          4.9888e-01,  1.1160e-01],
        [-5.4051e-01, -1.7229e-01,  7.1834e-01,  ...,  5.0166e-01,
          4.2534e-01,  1.1147e-01]])